<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-30 13:39:55--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11821 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.54K  --.-KB/s    in 0s      

2022-06-30 13:39:55 (73.7 MB/s) - ‘utils.py’ saved [11821/11821]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [5]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [6]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [7]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [8]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [9]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [10]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
519/519 [==============================] - 744s 1s/step - loss: 14.6355 - auc: 0.7865 - accuracy: 0.6959 - val_loss: 0.9146 - val_auc: 0.8010 - val_accuracy: 0.7251
Epoch 2/15
519/519 [==============================] - 232s 445ms/step - loss: 10.1488 - auc: 0.8706 - accuracy: 0.7862 - val_loss: 0.7383 - val_auc: 0.8620 - val_accuracy: 0.7704
Epoch 3/15
519/519 [==============================] - 232s 445ms/step - loss: 6.9866 - auc: 0.9269 - accuracy: 0.8540 - val_loss: 0.8817 - val_auc: 0.8445 - val_accuracy: 0.7792
Epoch 4/15
519/519 [==============================] - 231s 442ms/step - loss: 5.1662 - auc: 0.9524 - accuracy: 0.8855 - val_loss: 0.9257 - val_auc: 0.8423 - val_accuracy: 0.7732
Epoch 5/15
519/519 [==============================] - 236s 453ms/step - loss: 4.4172 - auc: 0.9606 - accuracy: 0.9008 - val_loss: 0.8334 - val_auc: 0.8602 - val_accuracy: 0.7966
Epoch 6/15
519/519 [==============================] - 231s 443ms/step - loss: 3.5321 - auc: 0.9697 - accuracy: 

In [11]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        initial_epoch=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 16/20
519/519 [==============================] - 236s 452ms/step - loss: 0.4110 - auc: 0.9980 - accuracy: 0.9913 - val_loss: 1.2083 - val_auc: 0.8333 - val_accuracy: 0.8219
Epoch 17/20
519/519 [==============================] - 231s 443ms/step - loss: 0.4228 - auc: 0.9983 - accuracy: 0.9899 - val_loss: 1.0204 - val_auc: 0.8537 - val_accuracy: 0.8350
Epoch 18/20
519/519 [==============================] - 232s 446ms/step - loss: 0.2677 - auc: 0.9994 - accuracy: 0.9928 - val_loss: 1.0773 - val_auc: 0.8451 - val_accuracy: 0.8268
Epoch 19/20
519/519 [==============================] - 235s 451ms/step - loss: 0.1851 - auc: 0.9994 - accuracy: 0.9951 - val_loss: 1.1830 - val_auc: 0.8341 - val_accuracy: 0.8293
Epoch 20/20
519/519 [==============================] - 231s 443ms/step - loss: 0.2296 - auc: 0.9988 - accuracy: 0.9938 - val_loss: 1.0842 - val_auc: 0.8433 - val_accuracy: 0.8308


In [12]:
!cp ./logs/checkpoints/RGB-model.h5 ./drive/MyDrive/checkpoints/rgb-model.h5